In [1]:
import numpy as np
import pandas as pd


In [2]:
train_df = pd.read_csv('train_data.csv')

In [3]:
test_df = pd.read_csv('test_data.csv')

In [4]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [6]:
# Tokenize sentences into words
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in train_df['tweet'].astype(str)]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save or load the model
w2v_model.save("word2vec_model.model")
# w2v_model = Word2Vec.load("word2vec_model.model")

# Get the word vectors
word_vectors = w2v_model.wv

In [7]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(str(sentence)) for sentence in train_df['tweet']])
X_test = np.array([vectorize(str(sentence)) for sentence in test_df['tweet']])

In [8]:
Y_train = train_df['decision']

In [9]:
Y_test = test_df['decision']

In [10]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(multi_class='multinomial')
clf.fit(X_train, Y_train)

/Users/chandrikamukherjee/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Y_pred = clf.predict(X_test)
print('Accuracy:', accuracy_score(Y_test, Y_pred))

Accuracy: 0.6420347021135913


In [12]:
precision = precision_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
print(precision, f1, recall)

0.6248772780953795 0.6204643949599078 0.6420347021135913


In [13]:
from sklearn.metrics import classification_report

In [14]:
report = classification_report(Y_test, Y_pred)

In [15]:
print(report)

              precision    recall  f1-score   support

          -1       0.51      0.23      0.31      8038
           0       0.60      0.61      0.60     13637
           1       0.68      0.81      0.74     22184

    accuracy                           0.64     43859
   macro avg       0.60      0.55      0.55     43859
weighted avg       0.62      0.64      0.62     43859

